Pneumonia, Cross entropy, Resnet18, Pretrain

In [1]:
!pip install libauc==1.2.0
!pip install medmnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=be29df57367a98630a33b0f99cb17d8f7a5ce14bceb0046d2d06ce70bbc25f3b
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
import libauc;
import numpy as np
import pandas as pd
from medmnist import BreastMNIST
from libauc.models import resnet18
from libauc.datasets import CIFAR10
from libauc.losses.auc import pAUCLoss  # default: SOPA
from libauc.losses.auc import pAUC_CVaR_Loss
from libauc.optimizers import SOPA
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler  # data resampling (for binary class)
from libauc.metrics import auc_roc_score
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
import random
import scipy
from scipy.ndimage import rotate
from scipy import ndimage
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import zoom

import torch 
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

import warnings
warnings.filterwarnings("ignore")
SEED=123

<ipython-input-2-1c53d203e95b>:19: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [3]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=28, crop_size=26, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([ 
                              transforms.Grayscale(num_output_channels=3),                                               
                              transforms.ToTensor(),
                              transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              ])
       self.transform_test = transforms.Compose([
                              transforms.Grayscale(num_output_channels=3),  
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                              ])
       
       # for loss function
       self.pos_indices = np.flatnonzero(targets==1)
       self.pos_index_map = {}
       for i, idx in enumerate(self.pos_indices):
           self.pos_index_map[idx] = i

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            idx = self.pos_index_map[idx] if idx in self.pos_indices else -1
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target, int(idx)

In [5]:
# general params
weight_decay = 5e-4
total_epoch = 50
decay_epochs = [20, 40]
batch_size = 64

# By default, we use one-way partial AUC loss (SOPA)
alpha = 0.  # a: min_tpr=0. This is fixed (for reference only)
beta = 0.1  # b: max_fpr=0.1

# By default, pAUCLoss calls SOPA in the backend
margin = 1.0
eta = 1e1 # learning rate for control negative samples weights

# sampling parameters
sampling_rate = 0.5

# paramaters
SEED = 123

In [6]:
train_npz=BreastMNIST(split="train", download=True)
val_npz=BreastMNIST(split="val", download=True)
test_npz=BreastMNIST(split="test", download=True)

100%|██████████| 559580/559580 [00:01<00:00, 304330.31it/s]

Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz


In [7]:
def gaussian_blur_2d(img):
    random.seed(SEED)
    sigma = random.uniform(0.1,0.9)
    blurred = gaussian_filter(img, sigma=sigma)
    return blurred

def x_flip(img):
    random.seed(SEED)
    flipped = np.flipud(img)
    return flipped

def y_flip(img):
    random.seed(SEED)
    flipped = np.flip(img,axis=1)
    return flipped

def zoom_xy(img, min_zoom, max_zoom):
    random.seed(SEED)
    zoom_factor = random.uniform(min_zoom, max_zoom)
    new_shape = (int(img.shape[0] * zoom_factor), int(img.shape[1] * zoom_factor))

    # Zoom in on the image using scipy.ndimage.zoom()
    zoomed_img = rotate(img, zoom_factor, reshape=False)

    # Crop the zoomed image to the original dimensions
    crop_x = int((zoomed_img.shape[1] - img.shape[1]) / 2)
    crop_y = int((zoomed_img.shape[0] - img.shape[0]) / 2)
    zoomed_img = zoomed_img[crop_y:crop_y+img.shape[0], crop_x:crop_x+img.shape[1]]

    return zoomed_img

def random_rotation_2d(img, min_angle, max_angle):
    """ Randomly rotate an image by a random angle (-max_angle, max_angle).

    Arguments:
    max_angle: `float`. The maximum rotation angle.

    Returns:
    rotated 3D image
    """
    random.seed(SEED)
    img_rot = np.zeros(img.shape)
    angle = random.uniform(min_angle, max_angle)
    if random.randint(1,100) > 50:
        #in half the cases, rotate left. in other half, rotate right.
        angle *= -1
        # Following lines would rotate on z and y axis as well, but not using them in this kernel
#        # rotate along z-axis
#        image2 = scipy.ndimage.interpolation.rotate(image1, angle, mode='nearest', axes=(0, 1), reshape=False)
#        # rotate along y-axis
#        image3 = scipy.ndimage.interpolation.rotate(image2, angle, mode='nearest', axes=(0, 2), reshape=False)

    # rotate along x-axis
    img_rot = ndimage.rotate(img, angle, reshape=False)
    return img_rot.reshape(img.shape)

def img_augment_2d(X_train,y_train):
      my_img=X_train
      my_label=y_train
      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = gaussian_blur_2d(img)
        my_img=np.append(my_img,np.expand_dims(img1,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)
     
      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = x_flip(img)
        my_img=np.append(my_img,np.expand_dims(img1,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)
      
      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = y_flip(img)
        my_img=np.append(my_img,np.expand_dims(img1,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)

      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = random_rotation_2d(img, 1, 10)
        my_img=np.append(my_img,np.expand_dims(img1,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)

      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = random_rotation_2d(img, 1, 10)
        img2=  zoom_xy(img, 0.9, 1.1)
        img3 = y_flip(img2)
        my_img=np.append(my_img,np.expand_dims(img3,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)
    
      for i in range(0,X_train.shape[0]):
        img=X_train[i]
        img1 = zoom_xy(img, 0.9, 1.1)
        my_img=np.append(my_img,np.expand_dims(img1,axis=0),axis=0)
      my_label=np.append(my_label,y_train,axis=0)
    
      return my_img,my_label

In [8]:
X_train=train_npz.imgs
y_train=train_npz.labels

X_val=val_npz.imgs
y_val=val_npz.labels

X_test=test_npz.imgs
y_test=test_npz.labels

In [9]:
X_train,y_train=img_augment_2d(X_train,y_train)

In [10]:
imratio = 0.5
generator = ImbalancedDataGenerator(shuffle=True, verbose=True, random_seed=0)
(train_images, train_labels) = generator.transform(X_train, y_train, imratio=imratio)
(eval_images, eval_labels) = generator.transform(X_val, y_val, imratio=imratio)
(test_images, test_labels) = generator.transform(X_test, y_test, imratio=0.5) 

trainSet = ImageDataset(train_images, train_labels)
evalSet = ImageDataset(eval_images, eval_labels)
testSet = ImageDataset(test_images, test_labels, mode='test')

sampler = DualSampler(trainSet, batch_size, sampling_rate=sampling_rate)
trainloader = torch.utils.data.DataLoader(trainSet, batch_size=batch_size,  sampler=sampler,  shuffle=False,  num_workers=1)
evalloader = torch.utils.data.DataLoader(evalSet, batch_size=batch_size,  shuffle=False,  num_workers=1)
testloader = torch.utils.data.DataLoader(testSet , batch_size=batch_size, shuffle=False, num_workers=1)

#SAMPLES: [3822], POS:NEG: [2793 : 1029], POS RATIO: 0.7308
#SAMPLES: [78], POS:NEG: [57 : 21], POS RATIO: 0.7308
#SAMPLES: [156], POS:NEG: [114 : 42], POS RATIO: 0.7308


In [11]:
set_all_seeds(SEED)
model = resnet18(pretrained=False, num_classes=2, last_activation=None) 
model = model.cuda()

# define loss & optimizer
# loss_fn = CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

epoch_decay = 0.003 # refers gamma in the paper

loss_fn = AUCMLoss()
lr=0.1
optimizer = PESG(model, 
                 loss_fn=loss_fn,
                 lr=lr, 
                 momentum=0.9,
                 margin=margin,
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)
# optimizer.update_lr(decay_factor=10)

In [13]:
print ('Start Training')
print ('-'*30)
test_best = 0
best_train_auc = 0 
train_list, test_list = [], []
for epoch in range(total_epoch):
    if epoch in decay_epochs:
       optimizer.update_lr(decay_factor=10)
            
    train_pred, train_true = [], []
    model.train() 
    for idx, (data, targets, index) in enumerate(trainloader):
        data, targets  = data.cuda(), targets.cuda()
        y_pred = model(data)
        y_prob = torch.sigmoid(y_pred)
        loss = loss_fn(y_prob, targets) # Notes: make index>0 for positive samples, and index<0 for negative samples
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_pred.append(y_prob.cpu().detach().numpy())
        train_true.append(targets.cpu().detach().numpy())

    # validation  
    model.eval()
    with torch.no_grad():    
        test_pred = []
        test_true = [] 
        for jdx, data in enumerate(evalloader):
            test_data, test_labels, index = data
            test_data = test_data.cuda()
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_labels.numpy())
      
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        train_auc_mean =  auc_roc_score(test_true, test_pred)[0]


        if best_train_auc < train_auc_mean:
            best_train_auc = train_auc_mean
            torch.save(model.state_dict(), 'breast_model.pt')

        print ('Epoch=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, train_auc_mean, best_train_auc ))  


Start Training
------------------------------
Epoch=0, Val_AUC=0.8997, Best_Val_AUC=0.8997
Epoch=1, Val_AUC=0.9098, Best_Val_AUC=0.9098
Epoch=2, Val_AUC=0.8914, Best_Val_AUC=0.9098
Epoch=3, Val_AUC=0.9181, Best_Val_AUC=0.9181
Epoch=4, Val_AUC=0.9424, Best_Val_AUC=0.9424
Epoch=5, Val_AUC=0.8830, Best_Val_AUC=0.9424
Epoch=6, Val_AUC=0.9056, Best_Val_AUC=0.9424
Epoch=7, Val_AUC=0.9181, Best_Val_AUC=0.9424
Epoch=8, Val_AUC=0.8881, Best_Val_AUC=0.9424
Epoch=9, Val_AUC=0.9081, Best_Val_AUC=0.9424
Epoch=10, Val_AUC=0.9215, Best_Val_AUC=0.9424
Epoch=11, Val_AUC=0.9098, Best_Val_AUC=0.9424
Epoch=12, Val_AUC=0.8956, Best_Val_AUC=0.9424
Epoch=13, Val_AUC=0.9482, Best_Val_AUC=0.9482
Epoch=14, Val_AUC=0.9006, Best_Val_AUC=0.9482
Epoch=15, Val_AUC=0.9081, Best_Val_AUC=0.9482
Epoch=16, Val_AUC=0.8822, Best_Val_AUC=0.9482
Epoch=17, Val_AUC=0.9265, Best_Val_AUC=0.9482
Epoch=18, Val_AUC=0.9499, Best_Val_AUC=0.9499
Epoch=19, Val_AUC=0.9039, Best_Val_AUC=0.9499
Reducing learning rate to 0.01000 @ T=1827!


In [15]:
# Testing
ckpt =  torch.load("breast_model.pt")
model.load_state_dict(ckpt)
model.eval()
best_val_auc = 0
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for jdx, (data, targets, _) in enumerate(testloader):
        test_data, test_labels = data, targets
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())

    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    test_auc_mean = auc_roc_score(test_true, test_pred)[0] 

    print ('Test result :::::::::  Test_AUC=%.4f, Best_train_AUC=%.4f'%(test_auc_mean, best_train_auc))

Test result :::::::::  Test_AUC=0.9010, Best_train_AUC=0.9499
